In [1]:
using DataFrames
using DataStructures
using CSV
using StatsPlots
using Distributions
using Interpolations
include("../../ThinkBayes.jl/src/ThinkBayes.jl")
using .ThinkBayes

In [60]:
df2011 = DataFrame(CSV.File("../../ThinkBayes2/data/showcases.2011.csv"));
df2012 = DataFrame(CSV.File("../../ThinkBayes2/data/showcases.2012.csv"));
df2011 = dropmissing(df2011)
df2012 = dropmissing(df2012)
df2011 = permutedims(df2011, 1)
df2012 = permutedims(df2012, 1)

,Column1,Showcase 1,Showcase 2,Bid 1,Bid 2,Difference 1,Difference 2
,String,String7,String7,String7,String7,String7,String7
1,Sep. 24,40811,62485,22000,40500,18811,21985
2,Sep. 25,31259,32972,10000,21000,21259,11972
3,Sep. 26,41943,24755,39500,22500,2443,2255
4,Sep. 27,24946,34665,21513,32000,3433,2665
5,Sep. 28,26257,48635,23420,35000,2837,13635
6,Oct. 1,49928,42621,36500,34000,13428,8621
7,Oct. 2,30744,22253,26500,18000,4244,4253
8,Oct. 3,37671,24251,42000,26000,-4329,-1749
9,Oct. 4,32255,45871,25000,29000,7255,16871


In [61]:
function stoi(df)
    for name in names(df)[2:end]
        df[!, name] = [parse(Float64, x[1:end]) for x in df[!, name]];
    end
end

stoi (generic function with 1 method)

In [62]:
stoi(df2011)

In [63]:
stoi(df2012)

In [64]:
df = vcat(df2011, df2012)
df = df[:, names(df)[2:end]]
first(df, 3)

,Showcase 1,Showcase 2,Bid 1,Bid 2,Difference 1,Difference 2
,Float64,Float64,Float64,Float64,Float64,Float64
1,50969.0,45429.0,42000.0,34000.0,8969.0,11429.0
2,21901.0,34061.0,14000.0,59900.0,7901.0,-25839.0
3,32815.0,53186.0,32000.0,45000.0,815.0,8186.0
